# Web scraping

In [1]:
import pandas as pd
import numpy as np
#pip install ntscraper
from ntscraper import Nitter

In [2]:
scraper = Nitter()

Testing instances: 100%|██████████| 30/30 [00:41<00:00,  1.37s/it]


## (1) Get tweets
Now we can get the tweets

In [3]:
tweets_food_hashtag=scraper.get_tweets("food",mode="hashtag",number=2000)

14-Dec-23 23:01:56 - No instance specified, using random instance https://nitter.tinfoil-hat.net
14-Dec-23 23:02:03 - Current stats for food: 15 tweets, 0 threads...
14-Dec-23 23:02:07 - Current stats for food: 26 tweets, 0 threads...
14-Dec-23 23:02:11 - Current stats for food: 32 tweets, 0 threads...
14-Dec-23 23:02:15 - Current stats for food: 39 tweets, 0 threads...
14-Dec-23 23:02:20 - Current stats for food: 50 tweets, 0 threads...
14-Dec-23 23:02:24 - Current stats for food: 64 tweets, 0 threads...
14-Dec-23 23:02:28 - Current stats for food: 71 tweets, 0 threads...
14-Dec-23 23:02:33 - Current stats for food: 80 tweets, 0 threads...
14-Dec-23 23:02:37 - Current stats for food: 84 tweets, 0 threads...
14-Dec-23 23:02:42 - Current stats for food: 92 tweets, 0 threads...
14-Dec-23 23:02:46 - Current stats for food: 104 tweets, 0 threads...
14-Dec-23 23:02:51 - Current stats for food: 114 tweets, 0 threads...
14-Dec-23 23:02:56 - Current stats for food: 120 tweets, 0 threads...
14-

In [59]:
len(tweets_food_hashtag) #2400

3192

### (1a) Small trials to discover tweets structure

In [124]:
print(type(tweets_food_hashtag))
for k in tweets_food_hashtag:
    print(k)
#type(tweets_food_hashtag["tweets"])

<class 'dict'>
tweets
threads


Threads is an empty list, since tweets seems interesting!

In [125]:
type(tweets_food_hashtag["tweets"])

list

It is a list...inspect its elements

In [126]:
tweets_food_hashtag["tweets"][2]

{'link': 'https://twitter.com/RujutaDiwekar/status/1732654796623196385#m',
 'text': 'Kaali gajar ki kanji-   Good for the gut, for the brain and for everything you care.  P.S. Video uses beetroot as kali gajar isn’t available everywhere.  #winter #food',
 'user': {'name': 'Rujuta Diwekar',
  'username': '@RujutaDiwekar',
  'profile_id': '670857437134479360',
  'avatar': 'https://pbs.twimg.com/profile_images/670857437134479360/QRM3YJd0_bigger.jpg'},
 'date': 'Dec 7, 2023 · 6:54 AM UTC',
 'is-retweet': True,
 'external-link': '',
 'quoted-post': {},
 'stats': {'comments': 22, 'retweets': 80, 'quotes': 2, 'likes': 531},
 'pictures': [],
 'videos': ['https://video.twimg.com/ext_tw_video/1732653764891557888/pu/vid/avc1/720x1280/Qzr0NEIpKreKrCBG.mp4?tag=12'],
 'gifs': []}

### (1b) Create a dataframe of tweets (and their info) 

In [60]:
final_tweets=[]
for tweet in tweets_food:
    useful_data=[ #tweet text and username
                  tweet['text'],
                  tweet['user']['username'],
                  #stats of the tweet
                  tweet['stats']['likes'],
                  tweet['stats']['comments'],
                  tweet['stats']['retweets'],
                  tweet['stats']['quotes'],
                  tweet['is-retweet'],
                  tweet['external-link'], #useful: an external link might have a bad influence on eng rate!
                  #maybe these vars will be encoded in a has/has not binary label
                  tweet['pictures'], 
                  tweet['videos'],
                  tweet['gifs']                 
                ]       
    final_tweets.append(useful_data)

In [71]:
tweets_df= pd.DataFrame(final_tweets,columns= ['text','username','likes','comments','retweets','quotes','is-retweet','external-link','pictures','videos','gifs'])
print("Df dims: ",tweets_df.shape)

Df dims:  (3192, 11)



NB: a differenza del progetto di indagine stat, qui non credo sia possibile ottenere verificato e 
lingua del tweet (anche se in realta su get_tweets posso filtrare per lingua....APPROFONDIRE!)

# Export dataset to CSV

In [73]:
# Remove duplicates
unique_tweets=tweets_df.drop_duplicates(subset=['text','username','likes','comments','retweets','quotes','is-retweet','external-link'],
                                        keep='first')

In [74]:
print("Df dims: ",unique_tweets.shape)

Df dims:  (1501, 11)


**NB: vedrete che dopo aver aggiunto le informazioni degli users (nell'altro file), il numero di righe del dataset è sceso a 1995 (alcuni utenti in questi giorni hanno messo il profilo privato/ hanno cancellato il profilo)**

In [75]:
unique_tweets.to_csv('../../datasets/tweets-data.csv', index=False)